<h1>User Defined Functions</h1>

A function definition starts with the keyword <b>function</b> followed by the function name and a set of parentheses followed by a colon :, and the body of the function is an indented block.


## First Example

In [15]:
function myPolynomial(x)
    """
    The argument x should be a number.
    Return 3*x+5
    """
    return 3*x+5
end

myPolynomial (generic function with 1 method)

In [2]:
myPolynomial(4)

17

In [3]:
myPolynomial(7+3)

35

**Notes**
<ul>
<li>The argument does not have to be named x, it can be any valid identifier.  Some functions have no argument, but you still need the parentheses after the function name.</li>
<li>The string enclosed in triple quotes at the top of the function definition is called the <b>docstring</b> and provides a short summary of what the function does.  The docstring should explain what argument types are expected and what is returned, if anything.</li>
<li>Notice that the body of the function definition is indented, like the bodies of for loops and if statements. Most languages require that a function definition body be enclosed in {..}, but Julia controls blocks of related code by indenting</li>
<li>This function returns a value that is a number.  A function can return any type of object, Any, Int, Float64, Array, AbstractString, Tuple, etc.  Some functions return nothing, they just perform some action, which can be typed with nothing.</li>
</ul>

The built-in ? operator that can be used to get help on a specific function or object.

Another way to get help in Julia is by using the @doc macro. The @doc macro allows you to access the documentation of a specific function or object.

In [34]:

? bswap


search: bswap



```
bswap(n)
```

Reverse the byte order of `n`.

(See also [`ntoh`](@ref) and [`hton`](@ref) to convert between the current native byte order and big-endian order.)

# Examples

```jldoctest
julia> a = bswap(0x10203040)
0x40302010

julia> bswap(a)
0x10203040

julia> string(1, base = 2)
"1"

julia> string(bswap(1), base = 2)
"100000000000000000000000000000000000000000000000000000000"
```


## Global Scope
For interactive use, the most common way of loading a module is <pre>using ModuleName </pre>

This loads the code associated with ModuleName, and brings:

1. the module name

2. and the elements of the export list into the surrounding global namespace.

Modules can introduce variables of other modules into their scope through the using or import statements or through qualified access using the dot-notation. 

Each module introduces a new global scope, separate from the global scope of all other modules—there is no all-encompassing global scope. 

In [1]:
using LinearAlgebra, Random

## Local Scope
A new local scope is introduced by most code blocks. If such a block is syntactically nested inside of another local scope, the scope it creates is nested inside of all the local scopes that it appears within, which are all ultimately nested inside of the global scope of the module in which the code is evaluated. 

In [6]:
x = 42

func = (x,y) -> x^y

func(3,2)

9

Explicit declaration works in Julia too: in any local scope, writing local x declares a new local variable in that scope, regardless of whether there is already a variable named x in an outer scope or not. 

In [13]:
function foo(n)
    x = 0
    for i in 1:n
        local x # loop-local x
        x = i
    end
    x # returns
end

foo(10)
        

0

When x = <value> occurs in a local scope, Julia applies the following rules to decide what the expression means based on where the assignment expression occurs and what x already refers to at that location:

1. **Existing local**: If x is already a local variable, then the existing local x is assigned;

   
2. **Hard scope**: If x is not already a local variable and assignment occurs inside of any hard scope construct
(i.e. within a let block, function, struct or macro body, comprehension, or generator), a new local named x is created in the scope of the assignment;


3. **Soft scope**: If x is not already a local variable and all of the scope constructs containing the assignment are soft scopes (loops, try/catch blocks), the behavior depends on whether the global variable x is defined:

<ul>
<li>if global x is undefined, a new local named x is created in the scope of the assignment;</li>
<li>if global x is defined, the assignment is considered ambiguous:</li>
    <ul>
<li>in non-interactive contexts (files, eval), an ambiguity warning is printed and a new local is created;</li>
<li>in interactive contexts (REPL, notebooks), the global variable x is assigned.</li>
    </ul>
</ul>

In [2]:
##########
# assignment inside of a hard scope
##########
function greet()
   x = "hello" # new local
   println(x)
end

greet() # the assignment occurs in local scope 

x

hello


LoadError: UndefVarError: `x` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

 there is no existing local x variable. Since x is local, it doesn't matter if there is a global named x or not. 

## Examples

Julia provides a standard Statistics library module that contains basic statistics functionality. you can invoke the library by writing <pre>using Statistics</pre> at the top of a julia program.


In [11]:
using Statistics


**median(itr)** compute the median of all elements in a collection

<b>For an even number of elements no exact median element exists, so the result is equivalent to calculating mean of two median elements</b>

In [86]:

println(median([1, 2, 3, 4]))
print(middle([1, 2, 3, 4]))

2.5
2.5

In [62]:
function udf_median(arr)
    """
    User defined median function
    """
    n = length(arr)
    sort!(arr,1)
    if n%2 == 1
        return arr[div(n,2, RoundDown)]
    else
        return (arr[n ÷ 2] + arr[n ÷ 2]) ÷ 2
    end
end

udf_median([1, 2, 3, 4])

2

In [40]:
? udf_median

search: udf_median median median!



No documentation found for private symbol.

`udf_median` is a `Function`.

```
# 1 method for generic function "udf_median" from Main:
 [1] udf_median(arr::AbstractArray)
     @ In[39]:1
```


In [41]:
udf_median([1,9,5,3])

3

In [55]:
using Random

Random.seed!(42)

x = rand(1, 10000)

typeof(x)

Matrix{Float64} (alias for Array{Float64, 2})

x is a list of 10,000  random boolean values. the middle should be 0.5 and median should round up towards 1.

In [56]:
# using Statistics
println(middle(x))
println(median(x))
udf_median(x)

0.4999670246992402
0.5025294562498319


LoadError: UndefKeywordError: keyword argument `dims` not assigned

In [96]:
typeof(udf_median)

typeof(udf_median) (singleton type of function udf_median, subtype of Function)

In [97]:
T = typeof(udf_median)
T <: Function

true

### stats()

The stats() function is going to return a tuple of statistics on a list of numbers:

minimum, maximum, mean, median.

For the min and max I will use the built-in functions minimum() and maximum().

I will compute the mean and I will get the median by calling the median() function.

In [15]:
function stats(thearr)
    """
    thearr should be a vector (or iterable) of numbers.
    Return a tuple minimum, maximum, mean, median
    """
    
    # Compute the mean
    mean = sum(thearr)/length(thearr)
    
    return minimum(thearr),maximum(thearr),mean,median(thearr)
end

stats (generic function with 3 methods)

In [110]:
stats([4,-1,12,7,2])

(-1, 12, 4.8, 4.0)

In [63]:
stats(x)

(2.5807448266190747e-6, 0.9999314686536538, 0.5015156416947841, 0.5025294562498319)

## Exercises

1. Write a function with signature **mean(arr)** that computes the mean of a array of numbers and **does not** use the built-in function sum().

2. Write a function with signature **eo(n)** that takes an integer argument and returns the string 'even' if n is even and returns 'odd' if n is odd.

In [72]:
function mean(arr)
    t = 0
    for i in arr
        t+=i
    end
    return t / length(arr)
end

mean(range(0,100))

50.0

In [73]:
function eo(n)
    if n % 2 == 0
        return "even"
    else
        return "odd"
    end
end

eo(-4)

"even"

## Dice Probabilities

In module 1.2 there was an exercise that asked you to approximate the probability of rolling either a 7 or an 11 with two dice.  Calculations like that are often done with functions.

The advantage of using a function is that the argument(s) can specify the event whose probability you want to approximate.  So one function can approximate many different probabilities.

The function twoDice() defined below is able to approximate the probability of many different events concerning the roll of two dice.  The function has one required argument that can be a array or iterable of numbers.  For example, twoDice(\[7,11]) will approximate the probability of rolling either a 7 or an 11 with two dice.

The function is going to test a roll to see if the event occurred using the **in** operator.  You can use in with arrays, sets, tuples, etc.  So we could also make the function call as twoDice({7,11}) or twoDice((7,11)).

There is an optional argument <b>trials</b> that is the number of times to repeat the experiment.  Trials is given a default value of 1000 so if the function call does not specify a value, trials is set to 1000.

In [87]:
using Random

function twoDice(x, ; trials=10000)
    """
    x should be an iterable, for example, a list or tuple, of roll outcomes
    that define an event.
    The optional argument trials is the number of times to run the experiment.
    Return an approximation of the probability of the event, total/trials.
   """

    total = 0
    for _ in 1:trials
        roll = sum(rand(1:6, 2))
        if roll in x
            total +=1
        end
    end
    return total/trials
end

twoDice (generic function with 2 methods)

In [89]:
Random.seed!(711)
twoDice([7,11])

0.2242

You can get a more accurate approximation by specifying a large number for trials.

In [88]:
Random.seed!(711)
twoDice([7,11], trials = 1000000)

0.222305

In [92]:
Random.seed!(1)
twoDice([2,3,4,5,6,7,8], trials = 10000)

0.7219

In [91]:
Random.seed!(1)

twoDice(range(2,9), trials = 10000)

0.8328

## Factorial

For an integer n greater than or equal to 1, <i>n factorial</i>, written $n!$, is defined by $n!=1\times2\times\cdots\times n$.

For example, $5! = 1\times 2\times 3\times 4\times 5=120$.

0! is defined to be 1.  Factorial is not defined for negative integers.

Here is a simple julia function that computes factorials.

In [98]:
function factorial1(n)
    """
    n should be a non-negative integer.
    Return n!, n factorial
    """
    total = 1

    for i in 2:n
        total*=i
    end
    return total
end

factorial1 (generic function with 1 method)

In [99]:
println(factorial1(0))
println(factorial1(5))
print(factorial1(10))

1
120
3628800

In [115]:
factorial1(25)

7034535277573963776

In [122]:
factorial1(π) # i dont know 

6.0

##### In Julia, exceeding the maximum representable value of a given type results in a wraparound behavior

In [119]:
print(typemax(Int64))
factorial1(26)

9223372036854775807

-1569523520172457984

####  For use cases where overflow cannot be tolerated under any circumstances, utilizing the BigInt type is advisable. 

In [121]:
factorial1(big(60))

8320987112741390144276341183223364380754172606361245952449277696409600000000000000

In [123]:
factorial1(-π)

1

**That is not good!**  The factorial of -3 is not defined and our function returned 1.

**What happened?**  The for loop is controlled by the range object range(2,n) so when n = -2, that becomes range(2,-2).  But range(2,-2) does not raise an exception, it is just an empty range, so the for loop does not execute, and the value total = 1 is returned.

To fix this problem we rewrite the factorial function and check for negative arguments.

## Raising Exceptions

In [124]:
function factorial2(n)
    """
    The argument n should be a non-negative integer.
    If n is negative, then raise a ValueError exception.
    Return n factorial.
    """
    if n < 0
        @error "n should be a non-negative intger"
    end

    total = 1
    for i in range(2,n)
        total *= i 
    end
    return total
end

factorial2(-2)

┌ Error: n should be a non-negative intger
└ @ Main In[124]:8


1

We are going to write one more version of the factorial function.

Instead of relying on range() to raise an exception when n is not an integer, we will have the function check that it has been given an integer argument.  That way we can supply an error message that says exactly what the problem is.

## isa function

In [127]:
?isa

search: isa isnan isdiag imag isreal ispath



```
isa(x, type) -> Bool
```

Determine whether `x` is of the given `type`. Can also be used as an infix operator, e.g. `x isa type`.

# Examples

```jldoctest
julia> isa(1, Int)
true

julia> isa(1, Matrix)
false

julia> isa(1, Char)
false

julia> isa(1, Number)
true

julia> 1 isa Number
true
```


In [138]:
println( isa(4, Float64))
println( isa(4, Int64))
println( isa("foo", String))
println( isa(π, Irrational))
println( isa(3.14, Float64))


false
true
true
true
true


In [139]:
function factorial3(n)
    if !isa(n, Int64)
        @error "n should be an integer"
    end
    if n < 0
        @error "n cannot be negative"
    end
    total = 1
    for i in range(2,n)
        total *= i
    end
    return total
end

factorial3(4.6)

┌ Error: n should be an integer
└ @ Main In[139]:3


24.0

In [141]:
factorial3(-2)

┌ Error: n cannot be negative
└ @ Main In[139]:6


1

In [140]:
factorial3(13)

6227020800

## Exercises

<ol>
    <li>Write a function with signature <b>mysum(x,y)</b> that takes two arguments and returns their sum.
        <ol>
            <li>Test mysum() by giving it two numerical arguments.</li>
            <li>What happens if you give mysum() two string arguments?</li>
            <li>What happens if you give mysum() a number and a string?</li>
        </ol>
    </li>
    <li>Write a function with signature <b>div(n)</b> that returns a list of positive divisors of the positive integer n.
        <ol>
            <li>At the beginning of the function body check that n is an integer and that it is positive.</li>
            <li>Note that in module 1.3.1, section 5.1, we had an example in which we found all the divisors of 36. This is similar, except it needs to handle any positive integer n.</li>
        </ol>
    </li>
    <li>If you have n items and you choose k of them, the number of ways to make this choice is called <b>n choose k</b> and the formula for it is ${n \choose k}=\frac{n!}{k!(n-k)!}$.<p>
        Example: ${10\choose 4}=\frac{10!}{4!(10-4)!}=\frac{10\cdot 9\cdot 8\cdot 7}{1\cdot 2\cdot 3\cdot 4}=210$.<p>
        Write a function with signature <b>choose(n,k)</b> that returns n choose k. You may use the function factorial3() defined earlier in this module.
    </li>
</ol>
    

